# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [29]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
from azureml.core.workspace import Workspace
#from azureml.core.script_run_config 
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.dataset import Dataset


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [30]:
ws = Workspace.from_config()
experiment_name = 'Udacity-exp-p3'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [31]:

# Specify parameter sampler
ps = RandomParameterSampling({
		'--C':choice(0.01,0.05, 0.1, 0.5,1),
		'--max_iter':choice(5, 10, 20, 50, 100)
	}
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2
,slack_factor=0.1
)

#if "training" not in os.listdir():
#    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory ='./',entry_script ='train.py', compute_target= 'aml-compute')
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,hyperparameter_sampling=ps, policy=policy,primary_metric_name="Accuracy",primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs= 10, max_concurrent_runs=4)


In [32]:
#TODO: Submit your experiment
hyperdrive = experiment.submit(config=hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [33]:
RunDetails(hyperdrive).show()
hyperdrive.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_92c06e04-1ee5-4667-9e30-a974882c976b
Web View: https://ml.azure.com/experiments/Udacity-exp-p3/runs/HD_92c06e04-1ee5-4667-9e30-a974882c976b?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-138655/workspaces/quick-starts-ws-138655

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-12T03:58:31.182459][API][INFO]Experiment created<END>\n""<START>[2021-02-12T03:58:31.670691][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-02-12T03:58:31.857640][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-02-12T03:58:31.8112694Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>
{}. Prepare run Id: HD_92c06e04-1ee5-4667-9e30-a974882c976b_preparation<END>"<START>[2021-02-12T03:59:03.286495][CONTROLLER][ERROR]Hyperdrive is unable to further process the experiment due to som

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Environment preparation run marked as Failed by execution service due to UserError.",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Environment preparation run marked as Failed by execution service due to UserError.\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
# 

best_run = hyperdrive.get_best_run_by_primary_metric()
best_run_metric = hyperdrive.get_metrics()
#Print('Accuracy:', best_run_metric['Accuracy'])
best_run_metric 

In [ ]:
best_run

In [ ]:
best_run_metric[best_run.id]

In [ ]:
# get name of files of best_run
best_run.get_file_names()

In [ ]:
#TODO: Save the best model

In [ ]:
# register model for future deployment
model = best_run.register_model(model_name='hyperdrive_model', model_path='/outputs/hyperdrivemodel.joblib')

In [ ]:
model.download(target_dir="outputs", exist_ok=True)
print('model downloaded')

In [ ]:
# Evaluation of model 
from train import clean_data
from azureml.data.dataset_factory import TabularDatasetFactory

factory = TabularDatasetFactory()
test_data_path = "https://raw.githubusercontent.com/nandex7/Capstone-Project---Azure-Machine-Learning-Engineer/main/data/WA_Fn-UseC_-Telco-Customer-Churn.csv"
test_ds = factory.from_delimited_files(test_data_path)
X_test, y_test = clean_data(test_ds)

logit_model = joblib.load('outputs/churn-model.joblib')

print(logit_model.score(X_test, y_test))

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service